In [ ]:
######################################################################################################
################# This is ML experiment  ##############################################
################# the training, testing are done for all BoW, tf-idf, randomforst ####################
######################################################################################################

## Import dependancies

In [ ]:
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 14.1 MB/s eta 0:00:00


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
#for import the data
import pandas as pd
import numpy as np

# for the visualisation
import matplotlib.pyplot as plt
import seaborn as sns

#for the text pre-processing (text cleaning)
import nltk
nltk.download('punkt')
import re #regular expression
import string
from nltk.tokenize import word_tokenize # word tokenization
from nltk.stem import PorterStemmer # word stemming

#for feature extraction ( vectorazation)
from sklearn.feature_extraction.text import CountVectorizer

#for split the data into train and test
from sklearn.model_selection import train_test_split

#for the model training
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

#for the model evaluation
from sklearn.metrics import multilabel_confusion_matrix,classification_report


from nltk.corpus import stopwords
from wordcloud import WordCloud
import contractions
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

## Import and preprocess dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/OCD/dataset/posts_with_ontology_labe_and_negative.csv")

In [ ]:
#Data with ontology and manual label
df_man = pd.read_csv('/content/drive/MyDrive/OCD/dataset/ontology_labeled_data_all_after_Embedding.csv')

In [ ]:
df.shape

(9497, 5)

In [ ]:
df.head()

post  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                Hi -\nSo I haven't been on here since December since I was recovering. I've been on prozac since I was 18 (now 24) for depression but I was diagnosed with OCD in January. My psych bumped me up to a higher dosage of prozac - 80mg - which was doing a great job. But I wanted to go off of it because it was making me gain weight and now I'm freaking out!! My OCD revolves around cheating like some of you know...and my most recent """"obsession"""" (I put quotes around it cuz i question whether its an obsession or if its real) is that... well let me start from the beginning. I went to my fiancé's friends place and we all had a few drinks and played card games and such. I was with my fiance the whole time except for when I went to the bathroom. I remember everything from the night despite the alcohol in my system. We came home and continued drinking (just me and my fiance) and I eventually blacked out. I ended up having a dream about cheating on my partner with someone that I had seen earlier that night (well not really cheating just taking the guys hands and placing them in an odd place). But it's just that image in my head nothing else. I don't really remember waking up from it to realize it was a dream, I think I stirred but then went right back to sleep (i dont even remember)...I keep thinking what if it wasn't a dream what if it actually happened? My fiance keeps telling me he was with me the whole time and nothing happened but I think about the dream/image all day and it's driving me nuts! I always question whether I re

In [ ]:
# defne new column named 'post_length' that contain the length of each post in the dataset
df["post"] = df["post"].astype(str)
df["post_length"]= df["post"].apply(len)

In [ ]:
print('the max length post is:',df['post_length'].max())
print('the min length post is:',df['post_length'].min())
print('the avg length post is:',df['post_length'].mean())

the max length post is: 10013
the min length post is: 1
the avg length post is: 1147.6772665052122


In [ ]:
# delete all posts that its length not in <2000 and >6
df = df[df.post_length<2000]
df = df[df.post_length>6]

In [ ]:
print(len(df))

7955


In [ ]:
# Check for NaN values and replace them with an empty string
df['post'] = df['post'].fillna('')

In [ ]:
import re
#define the process of text cleaning
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r' ',text)
#Clean Text
def clean_text(data):
    # convert catacter to lowercase
    data['clean_text']=data['post'].str.lower()
    #remove URLS
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r"http\S+", "", elem))
    #remove ponctuation
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r"[^\w\s]", "", elem))
    #remove
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'/n',"",elem))
    #remove degits
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'\d+',"",elem))
    #remove emojis
    data['clean_text'] = data['clean_text'].apply(lambda elem:deEmojify(elem))
    #remove multiple spaces
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'\s+'," ",elem))
    #remove single caracter
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'\s+[a-zA-Z]\s+'," ",elem))
    return data


def process_text(text):
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()

    # Handle contractions using the contractions library
    expanded_text = contractions.fix(text)

    # Lowercasing
    expanded_text = expanded_text.lower()

    # Tokenization
    tokens = word_tokenize(expanded_text)

    # Removing Punctuation
    tokens = [word for word in tokens if word.isalnum()]

    # Removing Stop Words
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return ' '.join(tokens)

In [ ]:
process_df = df.copy()
process_df

In [ ]:
#apply the process of cleaning for the train and test data
process_df = clean_text(process_df)
process_df['post'] = process_df['post'].apply(process_text)
#process_df['clean_text'] = process_df['clean_text'].apply(process_text)

In [ ]:
process_df.head()

,post,prefLabel,compulsion,obsession,expand1,post_length,clean_text
3,hello everyone could really use help right identifying whether another ocd obsession mind corrupt hav suffered ocd obsession harming family worst experience life get treatment form cbt helped however level exam stressfull ocd returned ob harm inappropriately touching people etc apart new obsession come whenever see young child laughing playing sitting usually 8 year old cousin visit u start intrusive thought negative experience childhood father alcoholic left panicky experience get really distressed thesehorrible reminder cry result avoid playing cousin cost dwell childhood feel depressed however seeing cousin brings horrible memory followed thought mind tell hopeless due childhood never amount anything childhood made way please help need know another ocd obsession find peace mind know bitter twisted idiot ocd,"['Treatment', 'hope', 'Thought', 'obsession', 'OCD']",0,1,"Aggressive obsession, Contamination obsession, Doubt obsession, Hoarding obsession, Religious obsession, Sexual obsession, Somatic obsession, Symmetry obsession, prophylactic treatment, therapeutic procedure, chemotherapy procedure, immunotherapy procedure, physical therapy procedure, radiation therapy procedure, transplantation, solid organ transplant, transfusion, morbid thoughts, Mental image, Homicidal thoughts, Persistent thoughts, Irrational thoughts, Suicidal ideation, Sexual thought, Aggressive thought, Contamination thought, Doubt thought, Hoarding thought, Obsessional thoughts, Religious thought, Somatic thought, Symmetry thought, Obsession",1345,hello everyone could really use your help right now in identifying whether this is another ocd obsession or just my mind being corrupt hav suffered before with ocd obsessions about harming my family and it was the worst experience of my life did get treatment in the form of cbt and this helped however after my level exams which were stressfull my ocd has returned and now obsess over harm inappropriately touching people etc apart from this new obsession has come whenever see any young child laughing or playing or just sitting usually my year old cousin who visits us start to have intrusive thought about negative experiences in my own childhood my father was an alcoholic which left me with very panicky experiences get really distressed by thesehorrible reminders and do cry as result avoid playing with my cousin at all costs so dont have to dwell on my own childhood and feel depressed however just seeing my cousin once brings these horrible memories which are followed by ongoing thoughts in my mind that tell me im hopeless due to my childhood will never amount to anything and my childhood made me this way please help me need to know if this is another ocd obsession so can find some peace of mind and know its not me being bitter and twisted idiot so is this ocd
6,better harm intrusive thought feel like falling got point realised thinking thing lie starting believe think going feel real like ill moment mostly revolve around mom cat something say going even think mom without getting gut wrenching feeling basically pounded mind ill also get blasphemous thought even hug think mean thing love mom hard feel much love enjoy time tired saw phychiatrist yesterday helpful waited month see decided sign therapy take forever feel helpless say fear could fear make feel like would horrible thing may even want really hope,"['Intrusive thoughts', 'fall', 'joy', 'hope', 'Thought', 'fear']",0,1,"Aggressive intrusive thoughts, Contamination intrusive thoughts, Doubt intrusive thoughts, Hoarding intrusive thoughts, Religious intrusive thoughts, Sexual intrusive thoughts, Somatic intrusive thoughts, Symmetry intrusive thoughts, morbid thoughts, Mental image, Homicidal thoughts, Persistent thoughts, Irrational thoughts, Suicidal ideation, Sexual thought, Aggressive thought, Contamination thought, Doubt thought, Hoarding thought, Obsessional thoughts, Religious thought

In [ ]:
# Assuming 'process_df' is the DataFrame name
# Print the number of posts
print(f"Number of posts: {len(process_df)}")

# Print the count of '1' in the 'obsession' column
print(f"Number of '1's in 'obsession' column: {process_df['obsession'].sum()}")
print(f"Number of '1's in 'compulsion' column: {process_df['compulsion'].sum()}")

# Print the count of '0's in both 'obsession' and 'compulsion' columns
print(f"Number of '0's in 'obsession' column: {len(process_df) - process_df['obsession'].sum()}")
print(f"Number of '0's in 'compulsion' column: {len(process_df) - process_df['compulsion'].sum()}")


Number of posts: 7955
Number of '1's in 'obsession' column: 3792
Number of '1's in 'compulsion' column: 1869
Number of '0's in 'obsession' column: 4163
Number of '0's in 'compulsion' column: 6086


## BoW and splitting the data

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Creating features using Bag of Words
text_data = process_df['post'].astype(str)

# Convert text to vectors using CountVectorizer with specified parameters
count_vect = CountVectorizer(max_features=25000, min_df=5, max_df=0.9, stop_words='english')
X = count_vect.fit_transform(text_data)

# Assuming 'compulsion' and 'obsession' are the target labels
Y = np.asarray(process_df[['compulsion', 'obsession']])


In [ ]:
# Get the shape (size) of the DTM
num_documents, num_tokens = X.shape

# Print the number of documents and tokens
print("Number of documents:", num_documents)
print("Number of unique tokens:", num_tokens)

Number of documents: 7955
Number of unique tokens: 4963


In [ ]:
print(X.shape)
print(Y.shape)

(7955, 4963)
(7955, 2)


In [ ]:
# Assuming X and Y are your feature and target variables
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(6364, 4963)
(1591, 4963)
(6364, 2)
(1591, 2)


## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix

# Assuming X_train, X_test, y_train, y_test are your training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
rf_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred_rf = rf_classifier.predict(X_test)

# Evaluate the model
conf_matrix = multilabel_confusion_matrix(y_test, y_pred_rf)

# Print confusion matrix for each class
for i, label in enumerate(['obsession', 'compulsion']):
    print(f"Confusion Matrix for {label}:")
    print(conf_matrix[i])
    print("\n")


Confusion Matrix for obsession:
[[1222    0]
 [  35  334]]


Confusion Matrix for compulsion:
[[788  44]
 [  9 750]]




In [ ]:
from sklearn.metrics import accuracy_score
# Assuming y_test and y_pred are your actual and predicted labels
label_names = ['obsession', 'compulsion']

# Calculate accuracy for each label
accuracy_per_label = {label: accuracy_score(y_test[:, i], y_pred_rf[:, i]) for i, label in enumerate(label_names)}

# Print accuracy for each label
for label, acc in accuracy_per_label.items():
    print(f'Accuracy for {label}: {acc}')


Accuracy for obsession: 0.9780012570710245
Accuracy for compulsion: 0.9666876178504086


In [ ]:
#classification report
label_names = ['obsession','compulsion']
report = classification_report(y_test, y_pred_rf, target_names=label_names)
print(report)

              precision    recall  f1-score   support

   obsession       1.00      0.91      0.95       369
  compulsion       0.94      0.99      0.97       759

   micro avg       0.96      0.96      0.96      1128
   macro avg       0.97      0.95      0.96      1128
weighted avg       0.96      0.96      0.96      1128
 samples avg       0.61      0.61      0.61      1128



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Prediction on manual label data

> Add blockquote



In [ ]:
unlabed_data = pd.read_csv('/content/drive/MyDrive/BERT_new_model/Dataset/ontology_labeled_data_all_after_Embedding.csv')

In [ ]:
unlabed_data.head()

,post,prefLabel,postAnnotations,mergedAnnotations,enriched-ontology-obsession,enriched-ontology-compulsion,obsession-expert,Compulsion-expert,Explanation
0,"January of this year I was let go from my job. My OCD had spiked and I was calling out sick for weeks. They were kind enough and felt bad I was so """"""""sick"""""""" that laid me off so I could collect unemployment and keep my insurance benefits. My unemployment runs out next week.\nI am doing much better now and though the prospect of going back to work scares me I do not have an option. My big concern is explaining the gap in my work history. It is not as though I can say, Hi, I have a mental illness and had a nervous breakdown.\nI am also concerned because I have talk therapy once to twice a week to deal with my depression and PTSD, and I am supposed to be starting CBT soon. I am not sure how I would go about getting the time needed for these very important things.\nDoes anyone have any advice to offer??\nThanks so much in advance!!\n~jenna","['OCD', 'depression']",['spike'],"['spike', 'depression', 'OCD']",1.0,NaN,1.0,0.0,My OCD had spiked and I was calling out sick for weeks. / I have a mental illness and had a nervous breakdown. / My unemployment runs out next week
1,"Hello everyone\nWe've had my two-year-old niece with us for most of this week and she's actually been really good for me because she's kept me busy and active and I think might have helped me confront some of my thoughts.\nHowever, right now, I'm a little bit frightened. I've been going through a thing for some time when I would keep checking all the switches in our hallway. Some of them don't 'go' anywhere, they're just stray switches but then because they don't do anything, that seemed to stress me out even more in case I started a fire somewhere secret. But more than that, I'm afraid I might have been checking for them for the sake of checking.\nThe truth is, I feel really upset and feel I'm setting too much store by my OCD right now, making myself sick and dizzy and hot by checking the switches night after night and I think I might have done it just because. I finally hit rock-bottom last night - there's one switch in particular that doesn't 'do anything', right next to the bathroom one. Both of them are outside the bathroom door. I keep checking both switches - even though it's obvious they're both turned off - and I keep checking and checking, counting to and fro in my head. I also get paranoid still about getting water on switches. Unfortunately, all this backwards and forwards kept making me want to go to the bathroom, which is a problem because it turns into a vicious circle.\nI got into a state last night and told my dad everything and he told me to 'rationalise' and not check the switches. So now I'm not.\nAnd this is the daft thing - but I feel really scared. I'm scared because I'm not doing it and I'm scared the game is finally up for me - that the whole world has done what it can for me and there's no more help I can be given. I'm scared because doing the rituals seem to have become part of me. I'm scared because I fear a kind of 'arrogance' taking me over and I'm scared of how I feel in case I sabotage my own chances of recovery - something I really fear doing.\nI feel I need a distraction, but I'm not sure what. I'm just so scared of myself.\nC x","['dizziness', 'Thought', 'hallway', 'OCD', 'outside', 'Rituals', 'fear', 'bathroom', 'checking']",['Rituals'],"['outside', 'hallway', 'bathroom', 'fear', 'Thought', 'Rituals', 'checking', 'dizziness', 'OCD']",NaN,1.0,0.0,1.0,"The truth is, I feel really upset and feel I'm setting too much store by my OCD right now, making myself sick and dizzy and hot by checking the switches night after night and I think I might have done it just because. I finally hit rock-bottom last night - there's one switch in particular that doesn't 'do anything', right next to the bathroom one. Both of them are outside the bathroom door. I keep checking both switches

In [ ]:
#apply the process of cleaning for the train and test data
unlabed_data = clean_text(unlabed_data)
unlabed_data['post'] = unlabed_data['post'].apply(process_text)
#process_df['clean_text'] = process_df['clean_text'].apply(process_text)
unlabed_data.head()

,post,prefLabel,postAnnotations,mergedAnnotations,enriched-ontology-obsession,enriched-ontology-compulsion,obsession-expert,Compulsion-expert,Explanation,clean_text
0,january year let go job ocd spiked calling sick week kind enough felt bad sick laid could collect unemployment keep insurance benefit unemployment run next week much better though prospect going back work scare option big concern explaining gap work history though say hi mental illness nervous breakdown also concerned talk therapy twice week deal depression ptsd supposed starting cbt soon sure would go getting time needed important thing anyone advice offer thanks much advance,"['OCD', 'depression']",['spike'],"['spike', 'depression', 'OCD']",1.0,NaN,1.0,0.0,My OCD had spiked and I was calling out sick for weeks. / I have a mental illness and had a nervous breakdown. / My unemployment runs out next week,january of this year was let go from my job my ocd had spiked and was calling out sick for weeks they were kind enough and felt bad was so sick that laid me off so could collect unemployment and keep my insurance benefits my unemployment runs out next week am doing much better now and though the prospect of going back to work scares me do not have an option my big concern is explaining the gap in my work history it is not as though can say hi have mental illness and had nervous breakdown am also concerned because have talk therapy once to twice week to deal with my depression and ptsd and am supposed to be starting cbt soon am not sure how would go about getting the time needed for these very important things does anyone have any advice to offer thanks so much in advance jenna
1,hello everyone niece u week actually really good kept busy active think might helped confront thought however right little bit frightened going thing time would keep checking switch hallway anywhere stray switch anything seemed stress even case started fire somewhere secret afraid might checking sake checking truth feel really upset feel setting much store ocd right making sick dizzy hot checking switch night night think might done finally hit last night one switch particular anything right next bathroom one outside bathroom door keep checking switch even though obvious turned keep checking checking counting fro head also get paranoid still getting water switch unfortunately backwards forward kept making want go bathroom problem turn vicious circle got state last night told dad everything told check switch daft thing feel really scared scared scared game finally whole world done help given scared ritual seem become part scared fear kind taking scared feel case sabotage chance recovery something really fear feel need distraction sure scared c x,"['dizziness', 'Thought', 'hallway', 'OCD', 'outside', 'Rituals', 'fear', 'bathroom', 'checking']",['Rituals'],"['outside', 'hallway', 'bathroom', 'fear', 'Thought', 'Rituals', 'checking', 'dizziness', 'OCD']",NaN,1.0,0.0,1.0,"The truth is, I feel really upset and feel I'm setting too much store by my OCD right now, making myself sick and dizzy and hot by checking the switches night after night and I think I might have done it just because. I finally hit rock-bottom last night - there's one switch in particular that doesn't 'do anything', right next to the bathroom one. Both of them are outside the bathroom door. I keep checking both switches - even though it's obvious they're both turned off - and I keep checking and checking, counting to and fro in my head. I also get paranoid still about getting water on switches. Unfortunately, all this backwards and forwards kept making me want to go to the bathroom, which is a problem because it turns into a vicious circle./ I'm scared because doing the rituals seem to have become part of me.\n",hello everyone weve had my twoyearold niece with us for most of this week and shes actually been really good for me because shes kept me busy and active and think might have helped me confront some of my thoughts howeve

In [ ]:
# Assuming 'clean_text' is the column containing text data
unlabeled_data_text = unlabed_data['post']

# Transform the unlabelled data using the same CountVectorizer
X1 = count_vect.transform(unlabeled_data_text)

# Make predictions using the loaded model
predictions = rf_classifier.predict(X1)

In [ ]:
# Assuming 'predictions' is a NumPy array or Pandas DataFrame containing the predicted labels
# Add these predictions to your unlabeled data or save them as needed
unlabed_data['predicted_compulsion'] = predictions[:, 0]
unlabed_data['predicted_obsession'] = predictions[:, 1]

# Display or save the results
print(unlabed_data[['post', 'predicted_compulsion', 'predicted_obsession']])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
from sklearn.metrics import classification_report, accuracy_score

# Assuming 'clean_text' is the column containing text data
unlabeled_data_vec_first_300 = count_vect.transform(unlabed_data['post'].head(300))

# Make predictions using the loaded model for the first 300 rows
predictions_first_300 = rf_classifier.predict(unlabeled_data_vec_first_300)

# Assuming unlabeled_data is your DataFrame for the first 300 rows
# Replace '...' with your actual DataFrame variable
unlabeled_data_first_300 = unlabed_data.head(300)

# Replace NaN values in the manual labels with 0 (you can choose a different approach)
y_true_first_300 = unlabeled_data_first_300[['obsession-expert', 'Compulsion-expert']].fillna(0).values

# Evaluate the model on the first 300 rows
print("\nEvaluation for 'obsession':")
print(classification_report(y_true_first_300[:, 0], predictions_first_300[:, 0]))

print("\nEvaluation for 'compulsion':")
print(classification_report(y_true_first_300[:, 1], predictions_first_300[:, 1]))

# Print accuracy
print("\nAccuracy for 'obsession':", accuracy_score(y_true_first_300[:, 0], predictions_first_300[:, 0]))
print("Accuracy for 'compulsion':", accuracy_score(y_true_first_300[:, 1], predictions_first_300[:, 1]))


Evaluation for 'obsession':
              precision    recall  f1-score   support

         0.0       0.39      1.00      0.56       114
         1.0       1.00      0.02      0.04       186

    accuracy                           0.39       300
   macro avg       0.69      0.51      0.30       300
weighted avg       0.77      0.39      0.24       300


Evaluation for 'compulsion':
              precision    recall  f1-score   support

         0.0       0.57      0.27      0.37       182
         1.0       0.38      0.69      0.49       118

    accuracy                           0.44       300
   macro avg       0.48      0.48      0.43       300
weighted avg       0.50      0.44      0.42       300


Accuracy for 'obsession': 0.3933333333333333
Accuracy for 'compulsion': 0.43666666666666665


In [ ]:
from sklearn.metrics import classification_report, accuracy_score

# Assuming df is your DataFrame containing the mentioned columns
# Replace this line with your actual DataFrame
df = ...

# Extract the true labels (manual labels) and predicted labels
y_true = unlabed_data[['obsession-expert', 'Compulsion-expert']].values
y_pred = unlabed_data[['predicted_obsession', 'predicted_compulsion']].values

# Evaluate the model on the first 300 posts (exclude rows without manual labels)
mask = ~unlabed_data['obsession-expert'].isnull()
y_true_sub = y_true[mask]
y_pred_sub = y_pred[mask]

# Count the number of manual labels for 'obsession'
obsession_manual_counts = unlabed_data['obsession-expert'].value_counts().to_dict()
print("\nManual Label Counts for 'obsession':")
print("Label 0:", obsession_manual_counts.get(0, 0))
print("Label 1:", obsession_manual_counts.get(1, 0))

# Count the number of manual labels for 'compulsion'
compulsion_manual_counts = unlabed_data['Compulsion-expert'].value_counts().to_dict()
print("\nManual Label Counts for 'compulsion':")
print("Label 0:", compulsion_manual_counts.get(0, 0))
print("Label 1:", compulsion_manual_counts.get(1, 0))

# Count the number of predicted labels for 'obsession'
obsession_pred_counts = unlabed_data['predicted_obsession'].value_counts().to_dict()
print("\nPredicted Label Counts for 'obsession':")
print("Label 0:", obsession_pred_counts.get(0, 0))
print("Label 1:", obsession_pred_counts.get(1, 0))

# Count the number of predicted labels for 'compulsion'
compulsion_pred_counts = unlabed_data['predicted_compulsion'].value_counts().to_dict()
print("\nPredicted Label Counts for 'compulsion':")
print("Label 0:", compulsion_pred_counts.get(0, 0))
print("Label 1:", compulsion_pred_counts.get(1, 0))



Manual Label Counts for 'obsession':
Label 0: 107
Label 1: 186

Manual Label Counts for 'compulsion':
Label 0: 177
Label 1: 118

Predicted Label Counts for 'obsession':
Label 0: 460
Label 1: 1309

Predicted Label Counts for 'compulsion':
Label 0: 1754
Label 1: 15


## NB

In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
#intialize the Random Forest Model
clf_NB = MultiOutputClassifier(MultinomialNB()).fit(X_train, y_train)
#train the model
clf_NB.fit(X_train,y_train)

MultiOutputClassifier(estimator=MultinomialNB())

In [ ]:
prediction_NB= clf_NB.predict(X_test)

In [ ]:
prediction_NB

array([[0, 1],
       [0, 0],
       [0, 1],
       ...,
       [1, 1],
       [0, 1],
       [1, 1]])

In [ ]:
#classification report
label_names = ['obsession','compulsion']
print(classification_report(y_test, prediction_NB,target_names=label_names))

              precision    recall  f1-score   support

   obsession       0.73      0.91      0.81       369
  compulsion       0.80      0.93      0.86       759

   micro avg       0.78      0.92      0.84      1128
   macro avg       0.77      0.92      0.84      1128
weighted avg       0.78      0.92      0.85      1128
 samples avg       0.55      0.59      0.56      1128



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
# Print confusion matrix
# Print confusion matrix
conf_matrix = multilabel_confusion_matrix(y_test, prediction_NB)
print("Confusion Matrix:")
print(conf_matrix)

#For the 'obsession' class:
#True Positive (TP):
#False Positive (FP):
#True Negative (TN):
#False Negative (FN):

#For the 'compulsion' class:
#True Positive (TP):
#False Positive (FP):
#True Negative (TN):
#False Negative (FN):

Confusion Matrix:
[[[1097  125]
  [  32  337]]

 [[ 659  173]
  [  53  706]]]


In [ ]:
from sklearn.metrics import classification_report, accuracy_score

# Calculate accuracy
accuracy = accuracy_score(y_test, prediction_NB)
print(f'Accuracy: {accuracy}')


Accuracy: 0.7743557510999372


In [ ]:

# Assuming y_test and y_pred are your actual and predicted labels
label_names = ['obsession', 'compulsion']

# Calculate accuracy for each label
accuracy_per_label = {label: accuracy_score(y_test[:, i], prediction_NB[:, i]) for i, label in enumerate(label_names)}

# Print accuracy for each label
for label, acc in accuracy_per_label.items():
    print(f'Accuracy for {label}: {acc}')


Accuracy for obsession: 0.9013199245757385
Accuracy for compulsion: 0.857950974230044


## Prediction on manual label using NB

In [ ]:
# Assuming 'clean_text' is the column containing text data
unlabeled_data_vec_first_300 = count_vect.transform(unlabed_data['clean_text'].head(300))

# Make predictions using the loaded model for the first 300 rows
predictions_first_300 = clf_NB.predict(unlabeled_data_vec_first_300)

# Assuming unlabeled_data is your DataFrame for the first 300 rows
# Replace '...' with your actual DataFrame variable
unlabeled_data_first_300 = unlabed_data.head(300)

# Replace NaN values in the manual labels with 0 (you can choose a different approach)
y_true_first_300 = unlabeled_data_first_300[['obsession-expert', 'Compulsion-expert']].fillna(0).values

# Evaluate the model on the first 300 rows
print("\nEvaluation for 'obsession':")
print(classification_report(y_true_first_300[:, 0], predictions_first_300[:, 0]))

print("\nEvaluation for 'compulsion':")
print(classification_report(y_true_first_300[:, 1], predictions_first_300[:, 1]))

# Print accuracy
print("\nAccuracy for 'obsession':", accuracy_score(y_true_first_300[:, 0], predictions_first_300[:, 0]))
print("Accuracy for 'compulsion':", accuracy_score(y_true_first_300[:, 1], predictions_first_300[:, 1]))


Evaluation for 'obsession':
              precision    recall  f1-score   support

         0.0       0.37      0.67      0.48       114
         1.0       0.61      0.32      0.42       186

    accuracy                           0.45       300
   macro avg       0.49      0.49      0.45       300
weighted avg       0.52      0.45      0.44       300


Evaluation for 'compulsion':
              precision    recall  f1-score   support

         0.0       0.53      0.42      0.47       182
         1.0       0.32      0.42      0.36       118

    accuracy                           0.42       300
   macro avg       0.42      0.42      0.42       300
weighted avg       0.45      0.42      0.43       300


Accuracy for 'obsession': 0.45
Accuracy for 'compulsion': 0.42


## K-Nearest Neighbors (KNN)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming X_train, X_test, y_train, y_test are your training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Initialize the KNN model with specified parameters
knn_model = KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto')

# Train the KNN model
knn_model.fit(X_train, y_train)

# Make predictions on the test set
knn_predictions = knn_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, knn_predictions)

print(f'Accuracy for KNN: {accuracy}')

Accuracy for KNN: 0.5971087366436204


In [ ]:
#classification report
label_names = ['obsession','compulsion']
print(classification_report(y_test, knn_predictions,target_names=label_names))

              precision    recall  f1-score   support

   obsession       0.74      0.19      0.30       369
  compulsion       0.90      0.51      0.66       759

   micro avg       0.88      0.41      0.56      1128
   macro avg       0.82      0.35      0.48      1128
weighted avg       0.85      0.41      0.54      1128
 samples avg       0.28      0.26      0.26      1128



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
# Print confusion matrix
# Print confusion matrix
conf_matrix = multilabel_confusion_matrix(y_test, knn_predictions)
print("Confusion Matrix:")
print(conf_matrix)

#For the 'obsession' class:
#True Positive (TP):
#False Positive (FP):
#True Negative (TN):
#False Negative (FN):

#For the 'compulsion' class:
#True Positive (TP):
#False Positive (FP):
#True Negative (TN):
#False Negative (FN):

Confusion Matrix:
[[[1198   24]
  [ 299   70]]

 [[ 791   41]
  [ 369  390]]]


In [ ]:
# Assuming y_test and y_pred are your actual and predicted labels
label_names = ['obsession', 'compulsion']

# Calculate accuracy for each label
accuracy_per_label = {label: accuracy_score(y_test[:, i], knn_predictions[:, i]) for i, label in enumerate(label_names)}

# Print accuracy for each label
for label, acc in accuracy_per_label.items():
    print(f'Accuracy for {label}: {acc}')

Accuracy for obsession: 0.7969830295411691
Accuracy for compulsion: 0.7423004399748586


In [ ]:
from sklearn.metrics import classification_report, accuracy_score

# Calculate accuracy
accuracy = accuracy_score(y_test, knn_predictions)
print(f'Accuracy: {accuracy}')


Accuracy: 0.5971087366436204


# TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf_vectorizer = TfidfVectorizer(max_features=3000, max_df=0.9)  # Adjust max_features as needed
X_tfidf = tfidf_vectorizer.fit_transform(process_df['post'])


# Assuming 'compulsion' and 'obsession' are the target labels
y = np.asarray(process_df[['compulsion', 'obsession']])

In [ ]:

# Split the data into training and testing sets
X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)


## Logistic regression using tf-idf

In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
# Re-fit the MultiOutputClassifier
clf_LR_tf = MultiOutputClassifier(LogisticRegression()).fit(X_train_tfidf, y_train)

# Example prediction
y_pred_tf = clf_LR_tf.predict(X_test_tfidf)


In [ ]:
label_names = ['obsession','compulsion']
print(classification_report(y_test, y_pred_tf,target_names=label_names))

              precision    recall  f1-score   support

   obsession       0.99      0.72      0.84       369
  compulsion       0.93      0.94      0.93       759

   micro avg       0.95      0.87      0.91      1128
   macro avg       0.96      0.83      0.89      1128
weighted avg       0.95      0.87      0.90      1128
 samples avg       0.56      0.56      0.56      1128



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
# Print confusion matrix
# Print confusion matrix
conf_matrix = multilabel_confusion_matrix(y_test, y_pred_tf)
print("Confusion Matrix:")
print(conf_matrix)

#For the 'obsession' class:
#True Positive (TP): 511
#False Positive (FP): 28
#True Negative (TN): 1394
#False Negative (FN): 67

#For the 'compulsion' class:
#True Positive (TP): 968
#False Positive (FP): 35
#True Negative (TN): 961
#False Negative (FN): 36


#True Positive (TP): Instances correctly predicted as belonging to the class.
#False Positive (FP): Instances incorrectly predicted as belonging to the class (they don't actually belong to it).
#True Negative (TN): Instances correctly predicted as not belonging to the class.
#False Negative (FN): Instances incorrectly predicted as not belonging to the class (they actually belong to it).


Confusion Matrix:
[[[1220    2]
  [ 102  267]]

 [[ 781   51]
  [  48  711]]]


In [ ]:
# Assuming y_test and y_pred are your actual and predicted labels
label_names = ['obsession', 'compulsion']

# Calculate accuracy for each label
accuracy_per_label = {label: accuracy_score(y_test[:, i], y_pred_tf[:, i]) for i, label in enumerate(label_names)}

# Print accuracy for each label
for label, acc in accuracy_per_label.items():
    print(f'Accuracy for {label}: {acc}')

Accuracy for obsession: 0.9346323067253299
Accuracy for compulsion: 0.9377749842866122


In [ ]:
from sklearn.metrics import classification_report, accuracy_score

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred_tf)
print(f'Accuracy: {accuracy}')


Accuracy: 0.8843494657448145


In [ ]:
import joblib

# Save the model and TF-IDF vectorizer
joblib.dump(clf_LR_tf, 'model_LR_tf.pkl')
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')


['tfidf_vectorizer.pkl']

In [ ]:
# Load the TF-IDF vectorizer
tfidf_vectorizer_loaded = joblib.load('tfidf_vectorizer.pkl')

# Transform the unlabeled data using the loaded TF-IDF vectorizer
unlabeled_data_tf = tfidf_vectorizer_loaded.transform(unlabed_data['clean_text'].head(293))

# Load the trained model
clf_LR_tf_loaded = joblib.load('model_LR_tf.pkl')

# Make predictions using the loaded model
predictions = clf_LR_tf_loaded.predict(unlabeled_data_tf)

In [ ]:
# Load the manual labels for 'obsession' and 'compulsion'
y_true_obsession = unlabed_data['obsession-expert']
y_true_compulsion = unlabed_data['Compulsion-expert']

# Drop rows with NaN values in the 'obsession_first_checker' column
y_true_obsession = y_true_obsession.dropna()

# Drop corresponding rows with NaN values in the 'compulsion_first_checker' column
y_true_compulsion = y_true_compulsion.dropna()


# Print the number of samples for investigation
print("Number of samples for 'y_true_obsession':", len(y_true_obsession))
print("Number of samples for 'predictions[:, 0]':", len(predictions[:, 0]))


Number of samples for 'y_true_obsession': 293
Number of samples for 'predictions[:, 0]': 293


In [ ]:
# Load the manual labels for 'obsession' and 'compulsion'
y_true_obsession = unlabed_data['obsession-expert'].head(293)
y_true_compulsion = unlabed_data['Compulsion-expert'].head(293)

# Print the number of samples for investigation
print("Number of samples for 'y_true_obsession':", len(y_true_obsession))
print("Number of samples for 'predictions[:, 0]':", len(predictions[:, 0]))

# Drop rows with NaN values in 'obsession_first_checker'
y_true_obsession = y_true_obsession.dropna()
y_true_compulsion = y_true_compulsion.dropna()

# Ensure that 'predictions[:, 0]' has the same length as 'y_true_obsession'
predictions_obsession = predictions[:len(y_true_obsession), 0]
predictions_compulsion= predictions[:len(y_true_compulsion), 0]

# Print the updated number of samples for investigation
print("Updated number of samples for 'y_true_obsession':", len(y_true_obsession))
print("Updated number of samples for 'predictions_obsession':", len(predictions_obsession))


Number of samples for 'y_true_obsession': 293
Number of samples for 'predictions[:, 0]': 293
Updated number of samples for 'y_true_obsession': 287
Updated number of samples for 'predictions_obsession': 287


In [ ]:
from sklearn.metrics import classification_report

# Evaluate the predictions for 'obsession'
print("\nEvaluation for 'obsession':")
print(classification_report(y_true_obsession, predictions_obsession))

print("\nEvaluation for 'compulsion':")
print(classification_report(y_true_compulsion, predictions_compulsion))


Evaluation for 'obsession':
              precision    recall  f1-score   support

         0.0       0.36      0.97      0.53       106
         1.0       0.00      0.00      0.00       181

    accuracy                           0.36       287
   macro avg       0.18      0.49      0.26       287
weighted avg       0.13      0.36      0.20       287


Evaluation for 'compulsion':
              precision    recall  f1-score   support

         0.0       0.60      0.99      0.75       173
         1.0       0.67      0.02      0.03       116

    accuracy                           0.60       289
   macro avg       0.63      0.51      0.39       289
weighted avg       0.63      0.60      0.46       289



In [ ]:
######################################################################################################
################# this is the end of experiment with ML ##############################################
################# the training, testing are done for all BoW, tf-idf, randomforst ####################
######################################################################################################

## Gradient Boosting using tf-idf:

In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Initialize the Gradient Boosting model
gb_model = GradientBoostingClassifier()

# Wrap the model with MultiOutputClassifier
multi_output_gb = MultiOutputClassifier(gb_model)

# Train the MultiOutputClassifier model
multi_output_gb.fit(X_train_tfidf, y_train)

# Make predictions on the test set
y_predictions_GB = multi_output_gb.predict(X_test_tfidf)

In [ ]:
label_names = ['obsession','compulsion']
print(classification_report(y_test, y_predictions_GB,target_names=label_names))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
# Print confusion matrix
# Print confusion matrix
conf_matrix = multilabel_confusion_matrix(y_test, y_pred_tf)
print("Confusion Matrix:")
print(conf_matrix)

#For the 'obsession' class:
#True Positive (TP): 511
#False Positive (FP): 28
#True Negative (TN): 1394
#False Negative (FN): 67

#For the 'compulsion' class:
#True Positive (TP): 968
#False Positive (FP): 35
#True Negative (TN): 961
#False Negative (FN): 36


#True Positive (TP): Instances correctly predicted as belonging to the class.
#False Positive (FP): Instances incorrectly predicted as belonging to the class (they don't actually belong to it).
#True Negative (TN): Instances correctly predicted as not belonging to the class.
#False Negative (FN): Instances incorrectly predicted as not belonging to the class (they actually belong to it).


## NB using tf-idf

In [ ]:
#intialize the Random Forest Model
clf_NB = MultiOutputClassifier(MultinomialNB()).fit(X_train_tfidf, y_train)
#train the model
prediction_NB_tf = clf_NB.predict(X_test_tfidf)

In [ ]:
label_names = ['obsession','compulsion']
print(classification_report(y_test, prediction_NB_tf,target_names=label_names))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
# Print confusion matrix
# Print confusion matrix
conf_matrix = multilabel_confusion_matrix(y_test, prediction_NB_tf)
print("Confusion Matrix:")
print(conf_matrix)

#For the 'obsession' class:
#True Positive (TP): 511
#False Positive (FP): 28
#True Negative (TN): 1394
#False Negative (FN): 67

#For the 'compulsion' class:
#True Positive (TP): 968
#False Positive (FP): 35
#True Negative (TN): 961
#False Negative (FN): 36


#True Positive (TP): Instances correctly predicted as belonging to the class.
#False Positive (FP): Instances incorrectly predicted as belonging to the class (they don't actually belong to it).
#True Negative (TN): Instances correctly predicted as not belonging to the class.
#False Negative (FN): Instances incorrectly predicted as not belonging to the class (they actually belong to it).


In [ ]:
# Assuming y_test and y_pred are your actual and predicted labels
label_names = ['obsession', 'compulsion']

# Calculate accuracy for each label
accuracy_per_label = {label: accuracy_score(y_test[:, i], prediction_NB_tf[:, i]) for i, label in enumerate(label_names)}

# Print accuracy for each label
for label, acc in accuracy_per_label.items():
    print(f'Accuracy for {label}: {acc}')

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

# Calculate accuracy
accuracy = accuracy_score(y_test, prediction_NB_tf)
print(f'Accuracy: {accuracy}')


## KNN + tf-idf

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Initialize the KNN model with specified parameters
knn_model = KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto')

# Train the KNN model
knn_model.fit(X_train_tfidf, y_train)

# Make predictions on the test set
knn_predictions_tf = knn_model.predict(X_test_tfidf)

# Calculate accuracy
accuracy = accuracy_score(y_test, knn_predictions_tf)

print(f'Accuracy for KNN: {accuracy}')

In [ ]:
#classification report
label_names = ['obsession','compulsion']
print(classification_report(y_test, knn_predictions_tf,target_names=label_names))

In [ ]:
# Assuming y_test and y_pred are your actual and predicted labels
label_names = ['obsession', 'compulsion']

# Calculate accuracy for each label
accuracy_per_label = {label: accuracy_score(y_test[:, i], knn_predictions_tf[:, i]) for i, label in enumerate(label_names)}

# Print accuracy for each label
for label, acc in accuracy_per_label.items():
    print(f'Accuracy for {label}: {acc}')

In [ ]:
conf_matrix = multilabel_confusion_matrix(y_test, knn_predictions_tf)
print("Confusion Matrix:")
print(conf_matrix)

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix


# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
rf_classifier.fit(X_train_tfidf, y_train)

# Make predictions on the test set
y_pred_rf_tf= rf_classifier.predict(X_test_tfidf)

# Evaluate the model
conf_matrix = multilabel_confusion_matrix(y_test, y_pred_rf_tf)

# Print confusion matrix for each class
for i, label in enumerate(['obsession', 'compulsion']):
    print(f"Confusion Matrix for {label}:")
    print(conf_matrix[i])
    print("\n")


In [ ]:
#classification report
label_names = ['obsession','compulsion']
print(classification_report(y_test, y_pred_rf_tf,target_names=label_names))

In [ ]:
# Assuming y_test and y_pred are your actual and predicted labels
label_names = ['obsession', 'compulsion']

# Calculate accuracy for each label
accuracy_per_label = {label: accuracy_score(y_test[:, i], y_pred_rf_tf[:, i]) for i, label in enumerate(label_names)}

# Print accuracy for each label
for label, acc in accuracy_per_label.items():
    print(f'Accuracy for {label}: {acc}')

## BOW using labeled data and manual label

In [ ]:
#Data with negative sample (without annotation)
df = pd.read_csv('/content/drive/MyDrive/post classification/processed_data/posts-ontology-manual-label.csv')

In [ ]:
df.head()

In [ ]:
df['post'] = df['post'].astype(str)
df = clean_text(df)
df['post'] = df['post'].apply(process_text)
#process_df['clean_text'] = process_df['clean_text'].apply(process_text)
df.head()

### BOW and LogisticRegression

In [ ]:
# Creating features using Bag of Word
text_data = df['post'].astype(str)

# Convert text to vectors using CountVectorizer
count_vect = CountVectorizer()
X = count_vect.fit_transform(text_data)

# Assuming 'compulsion' and 'obsession' are the target labels
Y = np.asarray(df[['compulsion', 'obsession']])

In [ ]:
#devide the data into train and test
# Get the shape (size) of the DTM
num_documents, num_tokens = X.shape

# Print the number of documents and tokens
print("Number of documents:", num_documents)
print("Number of unique tokens:", num_tokens)
print(X.shape)
print(Y.shape)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report

# Assuming X and Y are your features and labels
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

# Handle NaN values in features using SimpleImputer
imputer_features = SimpleImputer(strategy='mean')
X_train_imputed = imputer_features.fit_transform(X_train)
X_test_imputed = imputer_features.transform(X_test)

# Handle NaN values in labels using SimpleImputer
imputer_labels = SimpleImputer(strategy='most_frequent')
y_train_imputed = imputer_labels.fit_transform(y_train)
y_test_imputed = imputer_labels.transform(y_test)

# Initialize the logistic regression model
clf_LR = MultiOutputClassifier(LogisticRegression())

# Fit the model to the training data
clf_LR.fit(X_train_imputed, y_train_imputed)

# Make predictions on the test set
predictions = clf_LR.predict(X_test_imputed)

# Evaluate the model
print(classification_report(y_test_imputed, predictions))
